# 네이버 환전 페이지를 파싱해보자

## 파싱/크롤링을 할 때 셀레늄이 꼭 필요한가?
- 그렇지 않음
- 셀레늄은 페이지가 동적으로 그려지는(ajax 등) 경우에 필요
- 일반적으로 정적인 페이지는 셀레늄이 없어도 가능 

In [1]:
import datetime
import urllib.request as req  # 웹 요청할 때 사용할 파이썬 기본 라이브러리
from bs4 import BeautifulSoup  # 일반적으로 돔 파싱할때 사용하는 라이브러리
import pandas as pd

## 파싱할 페이지의 html을 가져오기

In [2]:
url = "https://finance.naver.com/marketindex/exchangeList.nhn"

data = req.urlopen(url)
html = data.read().decode("euc-kr")  # 

## HTML 파서 지정

In [3]:
soup = BeautifulSoup(html, 'html.parser')

## 웹 데이터 긁어오기

In [4]:
thead = soup.select_one("table > thead")  # 데이터의 헤더인 것 같다
table_data = soup.select("table > tbody > tr")  # html 테이블의 실제 데이터가 담긴 곳들
now_time = datetime.datetime.now().strftime("%Y.%m.%d %H%M%S")

# 테이블 헤더를 보면 알겠지만 특정 컬림이 분리되어있어서 데이터와 헤더가 맞지 않음
# 이를 해결하기 위해 별도의 헤더를 리스트로 생성해 줬음
head = ["통화명", "매매기준율", "현찰/사실 때", "현찰/파실 때", "송금/보내실 때", "송금/받으실 때", "미화환산율"]
body = []

# 실제 테이블 내용을 순회하면서 필요한 데이터만 빼옴
for row in table_data:
    new_row = [i.text.strip() for i in row.select("td")]
    body.append(new_row)

print(now_time)
print(head)
print(body)

2020.07.31 095726
['통화명', '매매기준율', '현찰/사실 때', '현찰/파실 때', '송금/보내실 때', '송금/받으실 때', '미화환산율']
[['미국 USD', '1,191.00', '1,211.84', '1,170.16', '1,202.60', '1,179.40', '1.000'], ['유럽연합 EUR', '1,413.24', '1,441.36', '1,385.12', '1,427.37', '1,399.11', '1.187'], ['일본 JPY (100엔)', '1,139.11', '1,159.04', '1,119.18', '1,150.27', '1,127.95', '0.956'], ['중국 CNY', '170.18', '178.68', '161.68', '171.88', '168.48', '0.143'], ['홍콩 HKD', '153.67', '156.69', '150.65', '155.20', '152.14', '0.129'], ['대만 TWD', '40.66', '45.98', '37.82', '0.00', '0.00', '0.034'], ['영국 GBP', '1,562.06', '1,592.83', '1,531.29', '1,577.68', '1,546.44', '1.312'], ['오만 OMR', '3,093.35', '3,368.65', '2,907.75', '0.00', '0.00', '2.597'], ['캐나다 CAD', '886.72', '904.18', '869.26', '895.58', '877.86', '0.745'], ['스위스 CHF', '1,310.95', '1,336.77', '1,285.13', '1,324.05', '1,297.85', '1.101'], ['스웨덴 SEK', '137.17', '140.53', '133.81', '138.54', '135.80', '0.115'], ['호주 AUD', '858.23', '875.13', '841.33', '866.81', '849.65', '0.721'], 

## 좀 더 보기 좋은 형태로 가공하기
- 주피터 노트북은 pandas 데이터, matplotlib 등의 데이터를 보기 좋게 정리해줌
- 이를 이용하여 깔끔한 형태의 데이터를 만들어 보자

In [5]:
df = pd.DataFrame(data=body, columns=head)
print(now_time)
df

2020.07.31 095726


,통화명,매매기준율,현찰/사실 때,현찰/파실 때,송금/보내실 때,송금/받으실 때,미화환산율
0,미국 USD,"1,191.00","1,211.84","1,170.16","1,202.60","1,179.40",1.000
1,유럽연합 EUR,"1,413.24","1,441.36","1,385.12","1,427.37","1,399.11",1.187
2,일본 JPY (100엔),"1,139.11","1,159.04","1,119.18","1,150.27","1,127.95",0.956
3,중국 CNY,170.18,178.68,161.68,171.88,168.48,0.143
4,홍콩 HKD,153.67,156.69,150.65,155.20,152.14,0.129
5,대만 TWD,40.66,45.98,37.82,0.00,0.00,0.034
6,영국 GBP,"1,562.06","1,592.83","1,531.29","1,577.68","1,546.44",1.312
7,오만 OMR,"3,093.35","3,368.65","2,907.75",0.00,0.00,2.597
8,캐나다 CAD,886.72,904.18,869.26,895.58,877.86,0.745
9,스위스 CHF,"1,310.95","1,336.77","1,285.13","1,324.05","1,297.85",1.101


## 파일로 저장하기

In [6]:
df.to_csv(now_time+".csv", sep=",", index=False)

## 저장된 파일 불러오기

In [7]:
new_df = pd.read_csv(now_time+".csv")
new_df

,통화명,매매기준율,현찰/사실 때,현찰/파실 때,송금/보내실 때,송금/받으실 때,미화환산율
0,미국 USD,"1,191.00","1,211.84","1,170.16","1,202.60","1,179.40",1.000
1,유럽연합 EUR,"1,413.24","1,441.36","1,385.12","1,427.37","1,399.11",1.187
2,일본 JPY (100엔),"1,139.11","1,159.04","1,119.18","1,150.27","1,127.95",0.956
3,중국 CNY,170.18,178.68,161.68,171.88,168.48,0.143
4,홍콩 HKD,153.67,156.69,150.65,155.20,152.14,0.129
5,대만 TWD,40.66,45.98,37.82,0.00,0.00,0.034
6,영국 GBP,"1,562.06","1,592.83","1,531.29","1,577.68","1,546.44",1.312
7,오만 OMR,"3,093.35","3,368.65","2,907.75",0.00,0.00,2.597
8,캐나다 CAD,886.72,904.18,869.26,895.58,877.86,0.745
9,스위스 CHF,"1,310.95","1,336.77","1,285.13","1,324.05","1,297.85",1.101


## DONE